### Imports

In [4]:
from LogicalQ.Logical import LogicalCircuit, LogicalStatevector, logical_state_fidelity
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Experiments import execute_circuits
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
from qiskit.transpiler import PassManager
from LogicalQ.Transpilation.UnBox import UnBoxTask

### Rx Gate

In [ ]:
qecc = steane_code
n, k, d = qecc['label']

lqcirc = LogicalCircuit(1, **qecc)

theta = 15.0

circ = QuantumCircuit(1)
circ.rx(theta, 0)

lqcirc.encode(0)
lqcirc.rx([0], theta)
lqcirc.measure([0], [0], with_error_correction=False)
#lqcirc.draw(output='mpl')

In [ ]:
result = execute_circuits(lqcirc, backend="aer_simulator", hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None, shots=1, save_statevector=True)[0]

l_sv = result.get_statevector()
lsv = LogicalStatevector(l_sv, lqcirc, k, qecc["label"], qecc["stabilizer_tableau"])

p_sv = Statevector(circ)

fidelity = logical_state_fidelity(lsv, p_sv)
fidelity

In [ ]:
lsv.draw(output='latex')

In [ ]:
p_sv.draw(output='latex')

### Ry Gate

In [ ]:
qecc = steane_code
n, k, d = qecc['label']

lqcirc = LogicalCircuit(1, **qecc)

theta = 15.0

circ = QuantumCircuit(1)
circ.ry(theta, 0)

lqcirc.encode(0)
lqcirc.ry([0], theta)
lqcirc.measure([0], [0], with_error_correction=False)
#lqcirc.draw(output='mpl')

In [ ]:
result = execute_circuits(lqcirc, backend="aer_simulator", hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None, shots=1, save_statevector=True)[0]

l_sv = result.get_statevector()
lsv = LogicalStatevector(l_sv, lqcirc, k, qecc["label"], qecc["stabilizer_tableau"])

p_sv = Statevector(circ)

fidelity = logical_state_fidelity(lsv, p_sv)
fidelity

In [ ]:
lsv.draw(output='latex')

In [ ]:
p_sv.draw(output='latex')

### Rz Gate

In [ ]:
qecc = steane_code
n, k, d = qecc['label']

lqcirc = LogicalCircuit(1, **qecc)

theta = 15.0

circ = QuantumCircuit(1)
circ.h(0)
circ.t(0)
circ.rz(theta, 0)

lqcirc.encode(0)
lqcirc.h(0)
lqcirc.t(0)
lqcirc.rz([0], theta, depth=20, recursion_degree=1)
lqcirc.measure([0], [0], with_error_correction=False)
#lqcirc.draw(output='mpl')

In [ ]:
#lqcirc.decompose().draw(output='mpl')

pm = PassManager([UnBoxTask()])
qc_unboxed = pm.run(lqcirc)

qc_unboxed.draw(output='mpl')

In [ ]:
result = execute_circuits(lqcirc, backend="aer_simulator", shots=1000, save_statevector=True)[0]#, hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None)[0]

l_sv = result.get_statevector()
lsv = LogicalStatevector(l_sv, lqcirc, k, qecc["label"], qecc["stabilizer_tableau"])
#counts = result.get_counts()
#lsv = LogicalStatevector.from_counts(counts, k, **qecc)

p_sv = Statevector(circ)

fidelity = logical_state_fidelity(lsv, p_sv)
fidelity

In [ ]:
lsv.draw(output='latex')

In [ ]:
p_sv.draw(output='latex')

### Rxx Gate

In [2]:
qecc = steane_code
n, k, d = qecc['label']

lqcirc = LogicalCircuit(2, **qecc)

theta = 15.0

circ = QuantumCircuit(2)
circ.rxx(theta, 0, 1)

lqcirc.encode([0, 1])
lqcirc.rxx(theta, [0, 1])
lqcirc.measure([0, 1], [0, 1], with_error_correction=False)
#lqcirc.draw(output='mpl')

In [5]:
pm = PassManager([UnBoxTask()])
qc_unboxed = pm.run(lqcirc)

qc_unboxed.draw(output='mpl')

qc_unboxed.count_ops()

OrderedDict([('h', 38),
             ('cx', 28),
             ('reset', 16),
             ('measure', 16),
             ('clogical.logicalop.z.transversal:$\\hat{Z}_{L}$', 15),
             ('clogical.logicalop.x.transversal:$\\hat{X}_{L}$', 10),
             ('x', 5),
             ('s', 2),
             ('sdg', 2),
             ('cclogical.logicalop.z.transversal:$\\hat{Z}_{L}$', 2),
             ('if_else', 2),
             ('t', 1),
             ('tdg', 1)])

In [ ]:
result = execute_circuits(lqcirc, backend="aer_simulator", shots=1, save_statevector=True)[0]#, hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None)[0]

In [9]:
l_sv = result.get_statevector()

In [ ]:
lsv = LogicalStatevector(l_sv, lqcirc, k, qecc["label"], qecc["stabilizer_tableau"])

In [ ]:
p_sv = Statevector(circ)

fidelity = logical_state_fidelity(lsv, p_sv)
fidelity

NameError: name 'l_sv' is not defined

Notes: Initializing LSV from Statevector is very slow. WIP to fix before Rxx gate validation.